<a href="https://colab.research.google.com/github/cdtlaura/nlp2/blob/main/finalregressionmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load and Explore the Dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Load the dataset
df = pd.read_csv('/content/Student_Performance.csv')

# Display the first few rows of the dataset
print(df.head())

# Display the column names
print(df.columns)

   Hours Studied  Previous Scores Extracurricular Activities  Sleep Hours  \
0              7               99                        Yes            9   
1              4               82                         No            4   
2              8               51                        Yes            7   
3              5               52                        Yes            5   
4              7               75                         No            8   

   Sample Question Papers Practiced  Performance Index  
0                                 1               91.0  
1                                 2               65.0  
2                                 2               45.0  
3                                 2               36.0  
4                                 5               66.0  
Index(['Hours Studied', 'Previous Scores', 'Extracurricular Activities',
       'Sleep Hours', 'Sample Question Papers Practiced', 'Performance Index'],
      dtype='object')


Preprocessing the Data

Handle Missing Values: We drop any rows with missing values for simplicity.
Encode Categorical Features: We use pd.get_dummies() to convert categorical variables like Make, Model, and Transmission Type into numerical format.

Feature Scaling: Since regression models benefit from having features on similar scales, we use StandardScaler to standardize the numerical features.

In [ ]:
# Handle missing values if necessary
df = df.dropna()

# Separate input features and target variable
X = df.drop(columns=['Performance Index'])  # MSRP is the target column
y = df['Performance Index']
# Encode categorical features
X_encoded = pd.get_dummies(X, drop_first=True)
print(X_encoded.head())  # These columns will be the input for the model - predictions
# Scale the numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

# Check the number of features
n_features = X_scaled.shape[1]


   Hours Studied  Previous Scores  Sleep Hours  \
0              7               99            9   
1              4               82            4   
2              8               51            7   
3              5               52            5   
4              7               75            8   

   Sample Question Papers Practiced  Extracurricular Activities_Yes  
0                                 1                            True  
1                                 2                           False  
2                                 2                            True  
3                                 2                            True  
4                                 5                           False  


Split the Data

In [ ]:
# Split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)

Define the Regression Model

Hidden Layers: We use two hidden layers with 64 and 32 neurons, both using ReLU activation.

Output Layer: The output layer has a single neuron to predict the car price (MSRP) with no activation function, which is typical for regression.

Loss Function: For regression tasks, we use mean squared error (MSE) as the loss function, which measures the difference between predicted and actual values.

In [ ]:
# Define the model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(n_features,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Single output neuron for regression (predicting price)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Train the Model

The model will iterate over the training data 100 times, learning to minimize the mean squared error.

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)


Evaluate the Model

This will print the mean squared error on the test data, which indicates how well the model is performing at predicting

In [ ]:
# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)
print('Test Mean Squared Error: %.2f' % loss)

Test Mean Squared Error: 4.41


Make Predictions

In [ ]:
# Create a sample input with all necessary features
# Make sure the sample matches the structure of X_encoded (use the same columns)
sample = pd.DataFrame({
    'Hours Studied': [7],
    'Previous Scores': [99],
    'Extracurricular Activities': [1],
    'Sleep Hours': [9],
    'Sample Question Papers Practiced': [2],
})

# Ensure the columns align with the training data (if columns differ, fill in missing ones with 0)
sample = sample.reindex(columns=X_encoded.columns, fill_value=0)

# Scale the sample data
sample_scaled = scaler.transform(sample)

# Predict the MSRP for the new car
prediction = model.predict(sample_scaled)
print('Predicted Performance Index: %.2f' % prediction[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted Performance Index: 91.16


<ipython-input-26-f3614ca42074>:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Predicted Performance Index: %.2f' % prediction[0])


Aha! (New Things Learned)

While I had prior experience with binary classification and regression, this assignment introduced me to new techniques and insights, especially in multiclass classification. A significant “aha!” moment was not only in implementing a multiclass classification model but also in learning how to interpret its results thoroughly and make necessary adjustments to improve accuracy. In particular, I discovered that modifying the structure of a neural network, such as increasing the number of layers, can enhance model accuracy. This hands-on experience in tweaking model architecture gave me a clearer understanding of how each component contributes to performance.

Huh? (Confusions)

Building the model posed several challenges, particularly when it came to finding the right balance in model architecture. For example, I found it confusing at times to determine how many layers and neurons to include in the neural network for the multiclass classification model. The trial-and-error process required to achieve an optimal configuration was time-consuming, and interpreting the impact of each change wasn’t straightforward. Additionally, while tuning the model, I occasionally encountered issues in understanding why certain adjustments didn’t yield expected improvements, which led to some confusion around the relationship between model complexity and performance.

Cool!

One of the most exciting parts of the project was seeing how adjusting the neural network’s structure directly impacted model accuracy. It was particularly rewarding to increase the number of layers in the multiclass model and observe a tangible improvement in performance. This felt empowering, as I could use my understanding of neural networks to make practical adjustments that enhanced the model’s predictive power. Knowing that these small changes had a measurable effect was motivating and validated the effort I put into fine-tuning the model.

Gotcha! (Changed Perspective)

Before this assignment, I viewed model building as a somewhat rigid process, where once a model type was chosen, the primary focus was just running it on data. This assignment shifted my perspective by showing me how much flexibility and control I have within each model, especially with neural networks, where the architecture itself can be fine-tuned to yield better results. This experience made me appreciate the iterative nature of model building and emphasized the importance of making deliberate choices in tuning model parameters and architecture, rather than relying on a one-size-fits-all approach.

Summary

Overall, this assignment taught me valuable lessons in multiclass classification and model fine-tuning. I’ve not only strengthened my understanding of binary classification and regression but have also developed skills in interpreting complex model results and making architectural adjustments to optimize performance. Applying these techniques in the future will allow me to adapt models to meet the specific requirements of each project. This experience has enhanced my ability to approach machine learning tasks more systematically, with a keen focus on model structure and performance tuning for improved outcomes.